I have a list of sentences and I have a search query. I want to find the most similar sentence in the list to the query sentence.

The query is: "**A well dressed man became a Member of Parliament.**"

The list of sentences to search is, in order of similarity:

1. "**Vel klæddur maður varð þingmaður.**" - the query sentence in Icelandic.
1. "**A well dressed man ran for office.**" - a similar sentence, dude becomes a politician.
1. "**The woman in the dress became a politician.**" - a slightly less similar sentence, a well dressed woman becomes a politician.
1. "**A well dressed man became a member of a football club.**" - a similar sentence with a different semantic meaning.

One common approach to this problem is to use a vector embedding model to represent each sentence as a vector, then use cosine similarity to compare them. The closer the vectors are, the more similar the sentences are. I covered this in a previous post: [Comparing AI embedding models](/posts/2025-01-25-comparing-embedding-models/).

In this post however, I am going to compare this similarity search with simply sending the query sentence along with the other sentences directly to the AI model and ask it to find the most similar sentence.

In [1]:
#| echo: false
import numpy as np
from openai import OpenAI
from sklearn.metrics.pairwise import cosine_similarity
import os
import great_tables as gt
import pandas as pd
import json

# Initialize the OpenAI client
# You'll need to set your API key in an environment variable or pass it directly
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

## Comparing the sentences using vector embeddings

I start by creating a function called `find_most_similar_sentence` (thanks Claude!). It takes a target sentence and a list of other sentences, and returns their similarity scores.

In [2]:
#| code-fold: true
def get_embedding(text, model="text-embedding-3-small"):
    """Get the embedding for a text using OpenAI's API"""
    response = client.embeddings.create(
        model=model,
        input=text,
        dimensions=1536  # Dimensionality of the embedding (default for text-embedding-3-small)
    )
    return response.data[0].embedding

def find_most_similar_sentence(target_sentence, sentence_list):
    """Find the sentence in sentence_list most similar to target_sentence"""
    # Get embeddings for all sentences
    target_embedding = get_embedding(target_sentence)
    sentence_embeddings = [get_embedding(sentence) for sentence in sentence_list]
    
    # Calculate cosine similarity between target and each sentence
    similarities = []
    for embedding in sentence_embeddings:
        # Reshape for sklearn's cosine_similarity function
        target_embedding_reshaped = np.array(target_embedding).reshape(1, -1)
        embedding_reshaped = np.array(embedding).reshape(1, -1)
        
        similarity = cosine_similarity(target_embedding_reshaped, embedding_reshaped)[0][0]
        similarities.append(similarity)
    
    # Find the index of the most similar sentence
    most_similar_index = np.argmax(similarities)
    
    return {
        "most_similar_sentence": sentence_list[most_similar_index],
        "similarity_score": similarities[most_similar_index],
        "all_similarities": list(zip(sentence_list, similarities))
    }


In [ ]:
sentences = [
    "A well dressed man ran for office.",
    "The woman in the dress became a politician.",
    "A well dressed man became a member of a football club.",
    "Vel klæddur maður varð þingmaður.", # the query sentence, but in Icelandic
]

query_sentence = "A well dressed man became a Member of Parliament."
result = find_most_similar_sentence(query_sentence, sentences)


If we put the results in a table and highlight closer matches in a stronger green, we can see the similarity scores more clearly:

In [4]:
# | code-fold: true

# Create DataFrame
df = pd.DataFrame(result['all_similarities'], columns=["Sentence", "Similarity Score"])


# Build a nice table
table = (
    gt.GT(df)
    .tab_header(
        title="Similarity Scores Using Vector Cosine Similarity",
        subtitle=f"Most similar sentence: to '{query_sentence}'"
    )
    .fmt_number(columns="Similarity Score", decimals=3)
    .data_color(
        columns="Similarity Score",
        palette="Greens",
        reverse=False
    )
)

table

GT(_tbl_data=                                            Sentence  Similarity Score
0                 A well dressed man ran for office.          0.676845
1        The woman in the dress became a politician.          0.536616
2  A well dressed man became a member of a footba...          0.611024
3                  Vel klæddur maður varð þingmaður.          0.242134, _body=<great_tables._gt_data.Body object at 0x166bde8a0>, _boxhead=Boxhead([ColInfo(var='Sentence', type=<ColInfoTypeEnum.default: 1>, column_label='Sentence', column_align='left', column_width=None), ColInfo(var='Similarity Score', type=<ColInfoTypeEnum.default: 1>, column_label='Similarity Score', column_align='right', column_width=None)]), _stub=<great_tables._gt_data.Stub object at 0x167fd8fe0>, _spanners=Spanners([]), _heading=Heading(title='Similarity Scores Using Vector Cosine Similarity', subtitle="Most similar sentence: to 'A well dressed man became a Member of Parliament.'", preheader=None), _stubhead=None, _source_notes=[], _footnotes=[], _styles=[StyleInfo(locname=LocBody(columns='Similarity Score', rows=[0]), grpname=None, colname='Similarity Score', rownum=0, colnum=None, styles=[CellStyleText(color='#FFFFFF', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#00441b')]), StyleInfo(locname=LocBody(columns='Similarity Score', rows=[1]), grpname=None, colname='Similarity Score', rownum=1, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#349e53')]), StyleInfo(locname=LocBody(columns='Similarity Score', rows=[2]), grpname=None, colname='Similarity Score', rownum=2, colnum=None, styles=[CellStyleText(color='#FFFFFF', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#077331')]), StyleInfo(locname=LocBody(columns='Similarity Score', rows=[3]), grpname=None, colname='Similarity Score', rownum=3, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#f7fcf5')])], _locale=<great_tables._gt_data.Locale object at 0x10f9e87a0>, _formats=[<great_tables._gt_data.FormatInfo object at 0x166bde8d0>], _substitutions=[], _options=Options(table_id=OptionsInfo(scss=False, category='table', type='value', value=None), table_caption=OptionsInfo(scss=False, category='table', type='value', value=None), table_width=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_layout=OptionsInfo(scss=True, category='table', type='value', value='fixed'), table_margin_left=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_margin_right=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_background_color=OptionsInfo(scss=True, category='table', type='value', value='#FFFFFF'), table_additional_css=OptionsInfo(scss=False, category='table', type='values', value=[]), table_font_names=OptionsInfo(scss=False, category='table', type='values', value=['-apple-system', 'BlinkMacSystemFont', 'Segoe UI', 'Roboto', 'Oxygen', 'Ubuntu', 'Cantarell', 'Helvetica Neue', 'Fira Sans', 'Droid Sans', 'Arial', 'sans-serif']), table_font_size=OptionsInfo(scss=True, category='table', type='px', value='16px'), table_font_weight=OptionsInfo(scss=True, category='table', type='value', value='normal'), table_font_style=OptionsInfo(scss=True, category='table', type='value', value='normal'), table_font_color=OptionsInfo(scss=True, category='table', type='value', value='#333333'), table_font_color_light=OptionsInfo(scss=True, category='table', type='value', value='#FFFFFF'), table_border_top_include=OptionsInfo(scss=False, category='table', 

The most similar sentence was correctly found, a well dressed man running for office, but the dude becoming a member of a football club gets a higher score than the woman becoming a politician.

The Icelandic sentence is at the bottom.

:::{.callout-note}
I might have gotten better results if I had used one of the multi-lingual embedding models. I'm not sure whether these are trained on Icelandic, and the research I did indicated that OpenAI's default models are pretty good at multilingual tasks. Try it for yourself and let me know.
:::

## Comparing the sentences using the AI model
Next, I ask the AI model directly which sentences were most similar, using this prompt:

> I will send you a list of sentences and you are to give me the list back along with a score of how closely they match my query sentence semantically, i.e. do they mean the same thing. 


In [6]:
# | code-fold: true

prompt = f"""I will send you a list of sentences and you are to give me the list back along with a score of how closely they match my query sentence semantically, i.e. do they mean the same thing. 

The query setneces is {query_sentence} and the list of sentences to compare it to are:
{sentences}

Return your result as a json object where
- "sentence" is the sentence form the list
- "score" is the score of how closely it matches the query sentence 
"""

response = client.chat.completions.create(
            model="gpt-4o",
            messages=[
                {"role": "system", "content": "You are a helpful assistant."},
                {"role": "user", "content": prompt},
            ],
            response_format={ "type": "json_object" }  # <- This ensures valid JSON is returned
)
        
data = json.loads(response.choices[0].message.content)
# Create DataFrame from the response
df2 = pd.DataFrame(data['results'])

table2 = (
    gt.GT(df2)
    .tab_header(
        title="Similarity Scores Using Direct Model Response",
        subtitle=f"Most similar sentence: to '{query_sentence}'"
    )
    .fmt_number(columns="score", decimals=3)
    .data_color(
        columns="score",
        palette="Greens"
    )
)

table2

GT(_tbl_data=                                            sentence  score
0                 A well dressed man ran for office.    0.7
1        The woman in the dress became a politician.    0.6
2  A well dressed man became a member of a footba...    0.4
3                  Vel klæddur maður varð þingmaður.    0.9, _body=<great_tables._gt_data.Body object at 0x166bdc560>, _boxhead=Boxhead([ColInfo(var='sentence', type=<ColInfoTypeEnum.default: 1>, column_label='sentence', column_align='left', column_width=None), ColInfo(var='score', type=<ColInfoTypeEnum.default: 1>, column_label='score', column_align='right', column_width=None)]), _stub=<great_tables._gt_data.Stub object at 0x1677f3e30>, _spanners=Spanners([]), _heading=Heading(title='Similarity Scores Using Direct Model Response', subtitle="Most similar sentence: to 'A well dressed man became a Member of Parliament.'", preheader=None), _stubhead=None, _source_notes=[], _footnotes=[], _styles=[StyleInfo(locname=LocBody(columns='score', rows=[0]), grpname=None, colname='score', rownum=0, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#4bb062')]), StyleInfo(locname=LocBody(columns='score', rows=[1]), grpname=None, colname='score', rownum=1, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#98d594')]), StyleInfo(locname=LocBody(columns='score', rows=[2]), grpname=None, colname='score', rownum=2, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#f7fcf5')]), StyleInfo(locname=LocBody(columns='score', rows=[3]), grpname=None, colname='score', rownum=3, colnum=None, styles=[CellStyleText(color='#FFFFFF', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#00441b')])], _locale=<great_tables._gt_data.Locale object at 0x167fd9df0>, _formats=[<great_tables._gt_data.FormatInfo object at 0x167fdbb90>], _substitutions=[], _options=Options(table_id=OptionsInfo(scss=False, category='table', type='value', value=None), table_caption=OptionsInfo(scss=False, category='table', type='value', value=None), table_width=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_layout=OptionsInfo(scss=True, category='table', type='value', value='fixed'), table_margin_left=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_margin_right=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_background_color=OptionsInfo(scss=True, category='table', type='value', value='#FFFFFF'), table_additional_css=OptionsInfo(scss=False, category='table', type='values', value=[]), table_font_names=OptionsInfo(scss=False, category='table', type='values', value=['-apple-system', 'BlinkMacSystemFont', 'Segoe UI', 'Roboto', 'Oxygen', 'Ubuntu', 'Cantarell', 'Helvetica Neue', 'Fira Sans', 'Droid Sans', 'Arial', 'sans-serif']), table_font_size=OptionsInfo(scss=True, category='table', type='px', value='16px'), table_font_weight=OptionsInfo(scss=True, category='table', type='value', value='normal'), table_font_style=OptionsInfo(scss=True, category='table', type='value', value='normal'), table_font_color=OptionsInfo(scss=True, category='table', type='value', value='#333333'), table_font_color_light=OptionsInfo(scss=True, category='table', type='value', value='#FFFFFF'), table_border_top_include=OptionsInfo(scss=False, category='table', type='boolean', value=True), table_border_top_style=OptionsInfo(scss=True, category='table', type='value', value='solid'), table_border_top_width=OptionsInfo(scss=True,

## Conclusion
The vector-based approach found the most similar sentence, but gave a false positive as the second closest match. It failed identifying the sentence that was a direct translation.

Sending the sentences direct to the LLM model worked better, it ordered the sentences in the right order and had the direct translation at the top.

The LLM method was a bit more fiddly - it took me a while to find the right prompt. I originally forgot to add "how closely they match semantically". This method is more expensive, but calling the current foundational models is still pretty cheap, so it depends on how many calls you have to make.